In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn
import torch.optim
import os
from torch.utils.data import Dataset, random_split, DataLoader
from midiutil import MIDIFile

cuda = torch.device('cuda:0')
cuda='cuda:0'

ModuleNotFoundError: No module named 'torch'

In [4]:
chord_types = {}

chord_map = {np.NaN: 'major',
    "''":'major',
'[]':'major',
'minor-seventh':'minor',
'dominant':'major',
'major':'major',
'minor':'minor',
'diminished':'minor',
'dominant-ninth':'major',
'major-sixth':'major',
'suspended-fourth':'major',
'major-seventh':'major',
'dominant-13th':'major',
'augmented':'minor',
'6':'minor',
'7':'major',
'maj':'major',
'min':'minor',
'half-diminished':'minor',
'diminished-seventh':'minor',
'major-ninth':'major',
'major-minor':'major',
'minor-ninth':'minor',
'minor-sixth':'minor',
'suspended-second':'major',
'minor-11th':'minor',
'dominant-11th':'major',
'augmented-seventh':'minor',
'min7':'minor',
'maj7':'major',
' dim7':'minor',
'nan':'major',
'dominant-seventh':'major',
'maj9':'major',
'dim':'minor',
'minor-major':'minor',
'power':'major',
'9':'major',
'minor-13th':'minor',
'augmented-ninth':'minor',
'maj69':'major',
'm7b5':'minor',
'aug':'minor',
'dim7':'minor',
'minMaj7':'minor',
'sus47':'major',
'pedal':'major',
'min9':"minor"}


        


In [5]:
def get_chord_types():
    for df in dataframes:
        for row in range(len(df["measure"])):
            if df["chord_type"][row] != '':
                chord_types[df["chord_type"][row]] = chord_types.get(df["chord_type"][row],0) + 1
    return chord_types

chord_types = get_chord_types()
for key in chord_types:
    print("'"+str(key)+"'")

notes = {"C":0,"D":2,"E":4,"F":5,"G":7,"A":9,"B":11}
note_map = {}
for note in notes:
    note_map[note+"0"] = notes[note]
    note_map[note+"b"] = (notes[note] - 1)%12
    note_map[note+"#"] = (notes[note] + 1)%12
    note_map[note+"2"] = (notes[note] + 1)%12 #weird but whatever
    note_map[note+"-2"] = (notes[note] + 1)%12 #weird but whatever

NameError: name 'dataframes' is not defined

In [4]:
def process_dataframe(df):
    for row_index in range(len(df["measure"])):
        num_measures = int(df["measure"][len(df["measure"])-1])
        current_measure = 1
        #row_index = 0
        this_measures_pitches = np.zeros(12)
        this_measures_chord = np.zeros(24)
        #let's just get all the data for all the measures. Then we'll go back and turn it into little 4-measure chunks
        melody_data = np.zeros((num_measures,12))
        chord_data = np.zeros((num_measures))
        bad_start = False
        if(df["chord_root"][0]) not in note_map:
            bad_start = True

        for row_index in range(len(df["measure"])):
            key_modifier = df["key_fifths"][row_index]*7
            if(df["chord_root"][row_index]) not in note_map:
                #print("bad row")
                assert current_measure == 1, "bad chord in measure " + str(current_measure) 
                continue
            if df["measure"][row_index] != current_measure or row_index == 0:
                #print("df['measure'][row_index] ",df["measure"][row_index])
                current_measure_str = df["measure"][row_index]
                if current_measure_str == "X1":
                    current_measure_str = "8" #weird case
                current_measure = int(current_measure_str) 
                chord_root_num = (note_map[df["chord_root"][row_index]] + key_modifier)%12
                chord_major = 0 if chord_map[df["chord_type"][row_index]] == "major" else 1 
                chord_num = 12*chord_major + chord_root_num
                chord_data[current_measure-1] = chord_num
            time_sig_mod = int(df["time"][row_index].split("/")[1]) / int(df["time"][row_index].split("/")[0])
            note_name = df["note_root"][row_index]
            if note_name != 'rest':
                note_dur = int(df["note_duration"][row_index])
                note_num = (note_map[note_name] + key_modifier)% 12
                melody_data[current_measure-1,note_num] += note_dur * time_sig_mod
    return melody_data, chord_data, bad_start

In [2]:
class ChordDataset(Dataset):
    def __init__(self, base_path):
        #time x batch x features but we don't worry about batch rn
        self.base_path = base_path
        self.training_paths = os.listdir(base_path)
        dataframes = []
        for path in self.training_paths:
            full_path = base_path + "/" + path
            dataframes.append(pd.read_csv(full_path))
            
        self.dataframes = dataframes
        self.measures_per_phrase = 4
        self.hop_size = 1
        self.note_map = note_map
        self.songs = []
        #self.song_lengths = []
        for df_num, df in enumerate(self.dataframes):
#             for row_index in range(len(df["measure"])):
#                 print(df["chord_root"][row_index])
            print("df_num: ", df_num)
            num_measures = int(df["measure"][len(df["measure"])-1])
            current_measure = 1
            #row_index = 0
            this_measures_pitches = np.zeros(12)
            this_measures_chord = np.zeros(24)
            #let's just get all the data for all the measures. Then we'll go back and turn it into little 4-measure chunks
            melody_data = np.zeros((num_measures,12))
            chord_data = np.zeros((num_measures))
            bad_start = False
            if(df["chord_root"][0]) not in self.note_map:
                bad_start = True
            
            for row_index in range(len(df["measure"])):
                key_modifier = df["key_fifths"][row_index]*7
                if(df["chord_root"][row_index]) not in self.note_map:
                    #print("bad row")
                    assert current_measure == 1, "bad chord in measure " + str(current_measure) 
                    continue
                if df["measure"][row_index] != current_measure or row_index == 0:
                    #print("df['measure'][row_index] ",df["measure"][row_index])
                    current_measure_str = df["measure"][row_index]
                    if current_measure_str == "X1":
                        current_measure_str = "8" #weird case
                    current_measure = int(current_measure_str) 
                    chord_root_num = (self.note_map[df["chord_root"][row_index]] + key_modifier)%12
                    chord_major = 0 if chord_map[df["chord_type"][row_index]] == "major" else 1 
                    chord_num = 12*chord_major + chord_root_num
                    chord_data[current_measure-1] = chord_num
                time_sig_mod = int(df["time"][row_index].split("/")[1]) / int(df["time"][row_index].split("/")[0])
                note_name = df["note_root"][row_index]
                if note_name != 'rest':
                    note_dur = int(df["note_duration"][row_index])
                    note_num = (self.note_map[note_name] + key_modifier)% 12
                    melody_data[current_measure-1,note_num] += note_dur * time_sig_mod
            if bad_start:
                self.songs.append((melody_data[1:], chord_data[1:]))
            else:
                self.songs.append((melody_data, chord_data))
            #self.song_lengths.append(num_measures)
        
        self.phrases = []
        for song_num in range(len(self.songs)):
            base_measure_num = 1
            num_measures = len(self.songs[song_num][0])
            while base_measure_num + self.measures_per_phrase <= num_measures: #still collecting datas
                melody_data_to_append = self.songs[song_num][0][base_measure_num-1:base_measure_num-1+self.measures_per_phrase]
                chord_data_to_append = self.songs[song_num][1][base_measure_num-1:base_measure_num-1+self.measures_per_phrase]
                self.phrases.append((torch.tensor(melody_data_to_append,dtype=torch.float32), torch.tensor(chord_data_to_append,dtype=torch.long)))
                base_measure_num += self.hop_size
         
    def __len__(self):
        return len(self.phrases)
    
    def __getitem__(self,idx):
        return self.phrases[idx]

    

NameError: name 'Dataset' is not defined

In [1]:
trainDataset = ChordDataset("csv_train")

testDatasetAll = ChordDataset("csv_test")



NameError: name 'ChordDataset' is not defined

In [157]:
num_test = int(len(testDatasetAll)*0.8)
testset, valset = random_split(testDatasetAll, [num_test, len(testDatasetAll)-num_test])

In [158]:
#NOTE: make it bidirectional girl 
class ChordLSTM(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, batch_size):
        super(ChordLSTM,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.input_size = input_size
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTM(input_size=self.input_size, hidden_size=self.hidden_size, num_layers=self.num_layers, dropout=0.2, batch_first=True, bidirectional=True).to(cuda)
        self.fc = torch.nn.Linear(self.hidden_size*2, 24).to(cuda)
        self.hidden = self.init_hidden()
        self.tanh = torch.nn.Tanh()
        self.loss_func = torch.nn.CrossEntropyLoss()
        #note: we don't want to reset hidden each time when we test this - want to feed data one at a time (faster)
    
    def init_hidden(self):
        return (torch.zeros(2*self.num_layers, self.batch_size, self.hidden_size).to(cuda), torch.zeros(2*self.num_layers, self.batch_size, self.hidden_size).to(cuda))
        
    def forward(self, input_sequence_batch):
        self.hidden = self.init_hidden()
        #print("input batch size ", input_sequence_batch.size())
        #print("hidden size ", self.hidden[0].size())
        lstm_out, self.hidden = self.lstm(input_sequence_batch, self.hidden)
        X = self.fc(lstm_out)
        X = self.tanh(X)
        return X #do sigmoid later 
    
    def loss(self, predictions, labels):
        #print("predictions size ", predictions.size())
        #print("labels size ", labels.size())
        pred = torch.transpose(predictions, 1, 2)
        return self.loss_func(pred, labels)

In [166]:
def train(num_epochs):
    batch_size = 512
    train_loader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last = True)
    val_loader = DataLoader(valset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True)
    model = ChordLSTM(12, 128, 2, batch_size)
    model.to(cuda)
    learning_rate = .0005
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    num_epochs = 300
    num_times_val_loss_decreased = 0
    prev_val_loss = np.inf
    for epoch in range(num_epochs):
        train_loss_sum = 0
        val_loss_sum = 0
        train_count = 0
        val_count = 0
        #for (x_padded,y_padded,x_lens,y_lens) in enumerate(train_loader):
        model.train()
        for (x,y) in train_loader:
            #print("shape of x ", x.size())
            model.zero_grad()
            output = model(x.to(cuda))
            loss = model.loss(output.to(cuda),y.to(cuda))
            train_loss_sum += loss
            loss.backward() #this might die... 
            optimizer.step()
            train_count += 1
        print("train_loss: ", train_loss_sum/train_count)
        
        #validate
        with torch.no_grad():
            model.eval()
            for (x,y) in val_loader:
                model.zero_grad()
                output = model(x.to(cuda))
                loss = model.loss(output.to(cuda),y.to(cuda))
                val_loss_sum += loss
                val_count += 1
            print("val_loss: ", val_loss_sum/val_count)
            if val_loss_sum/val_count > prev_val_loss:
                num_times_val_loss_decreased += 1
                print("num_times_val_loss_decreased ", num_times_val_loss_decreased)
            else:
                torch.save({"epoch":epoch, "model_state_dict":model.state_dict(),"optimizer_state_dict":optimizer.state_dict(),"loss":val_loss_sum/val_count}, "checkpoints/checkpoint.pth.tar")
                prev_val_loss = val_loss_sum/val_count
                num_times_val_loss_decreased = 0 #for 10 times in a row failing 
            if num_times_val_loss_decreased > 20:
                print("EARLY STOPPING")
                break


train(300)


train_loss:  tensor(2.5576, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.3623, device='cuda:0')
train_loss:  tensor(2.2802, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2767, device='cuda:0')
train_loss:  tensor(2.2292, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2498, device='cuda:0')
train_loss:  tensor(2.2049, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2401, device='cuda:0')
train_loss:  tensor(2.1864, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2298, device='cuda:0')
train_loss:  tensor(2.1714, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2244, device='cuda:0')
train_loss:  tensor(2.1589, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2236, device='cuda:0')
train_loss:  tensor(2.1472, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2129, device='cuda:0')
train_loss:  tensor(2.1369, device='cuda:0', grad_fn=<DivBackward0>)
val_loss:  tensor(2.2197, device='c

In [169]:
model = ChordLSTM(12, 128, 2, 1)
model.load_state_dict(torch.load("checkpoints/checkpoint.pth.tar")["model_state_dict"])
model.eval()


ChordLSTM(
  (lstm): LSTM(12, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=256, out_features=24, bias=True)
  (tanh): Tanh()
  (loss_func): CrossEntropyLoss()
)

In [183]:
mel, targets = testset[50]
inp = torch.zeros((1,4,12)).to(cuda)
inp[0,:,:] = mel
sm = torch.nn.Softmax(dim=2)
output_guess = sm(model(inp))
output_maxes = torch.max(output_guess,axis=2)
print("output_maxes ", output_maxes)
print(output_guess)
print(targets)

output_maxes  torch.return_types.max(
values=tensor([[0.2028, 0.2265, 0.2366, 0.2226]], device='cuda:0',
       grad_fn=<MaxBackward0>),
indices=tensor([[ 5, 10, 10, 10]], device='cuda:0'))
tensor([[[0.0274, 0.0274, 0.0275, 0.0537, 0.0274, 0.2028, 0.0274, 0.0274,
          0.0275, 0.0274, 0.0484, 0.0274, 0.1457, 0.0274, 0.0278, 0.0275,
          0.0274, 0.0275, 0.0274, 0.0275, 0.0274, 0.0275, 0.0274, 0.0274],
         [0.0307, 0.0307, 0.0307, 0.0522, 0.0307, 0.0336, 0.0307, 0.0307,
          0.0307, 0.0307, 0.2265, 0.0307, 0.0318, 0.0307, 0.0308, 0.0309,
          0.0307, 0.0307, 0.0307, 0.0730, 0.0307, 0.0307, 0.0307, 0.0307],
         [0.0320, 0.0320, 0.0320, 0.0459, 0.0320, 0.0336, 0.0320, 0.0320,
          0.0320, 0.0320, 0.2366, 0.0320, 0.0334, 0.0320, 0.0323, 0.0322,
          0.0320, 0.0320, 0.0320, 0.0416, 0.0320, 0.0320, 0.0320, 0.0320],
         [0.0302, 0.0301, 0.0302, 0.0522, 0.0301, 0.0310, 0.0301, 0.0302,
          0.0301, 0.0301, 0.2226, 0.0301, 0.0303, 0.0301, 0.0354, 0

In [260]:
test_songs_listed = os.listdir("csv_test")
df_to_test = pd.read_csv("csv_test/" + test_songs_listed[50])
df_to_test_melody, df_to_test_chords, bad_start = process_dataframe(df_to_test)
if bad_start:
    df_to_test_melody = df_to_test_melody[1:]
    df_to_test_chords = df_to_test_chords[1:]
df_to_test_melody = torch.tensor(df_to_test_melody).to(cuda)
df_to_test_chords = torch.tensor(df_to_test_chords).to(cuda)
print(df_to_test_melody, df_to_test_chords)

base_measure_num = 1
measures_per_phrase = 4
hop_size = 4
num_measures = len(df_to_test_melody)
output = []
while base_measure_num + measures_per_phrase <= num_measures: #run lstm every 4 measures
    melody_tensor = torch.zeros((1,4,12),dtype=torch.float32).to(cuda)
    melody_tensor[0,:,:] += df_to_test_melody[base_measure_num-1:base_measure_num-1+measures_per_phrase]
    #chord_data_to_append = self.songs[song_num][1][base_measure_num-1:base_measure_num-1+self.measures_per_phrase]
    model_out = model(melody_tensor)
    output.append(torch.max(model_out,axis=2)[1])
    base_measure_num += hop_size
print("output: ", output)
final_output = np.zeros(len(output)*4)
for i,o in enumerate(output):
    final_output[i*4:i*4+4] = o.cpu()
print("guess: ", final_output)
print("gt: ", np.array(df_to_test_chords.cpu()))

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 16.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 10.6667,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 16.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000, 10.6667,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 16.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000, 10.6667,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, 16.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000, 10.6667,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.

In [263]:
print(test_songs_listed[50])

chord_template_minor = np.array([0,3,7])
chord_template_major = np.array([0,4,7])
def write_midi(df, chords):
    MyMIDI = MIDIFile(2)
    MyMIDI.addTempo(0, 0, 100)
    MyMIDI.addTempo(1, 0, 100)
    current_location = 0
    num_time_sig = int(df["time"][0].split("/")[0])
    denom_time_sig = int(df["time"][1].split("/")[0])
    MyMIDI.addTimeSignature(0,0,num_time_sig,2,24)
    MyMIDI.addTimeSignature(1,0,num_time_sig,2,24)
    print(df["time"][0])
    for ri, row in enumerate(df["measure"]):
        row_index = ri
        time_sig_mod = int(df["time"][row_index].split("/")[1]) / int(df["time"][row_index].split("/")[0]) 
        time_sig_mod = 1/(denom_time_sig)
        print("mod ", time_sig_mod)
        beat_duration = float(df["note_duration"][row_index])*time_sig_mod
        if df["note_root"][row_index] == "rest":
            current_location += beat_duration
            continue
        midi_num = (note_map[df["note_root"][row_index]] + int(df["key_fifths"][row_index])*7)%12 + 12*int(df["note_octave"][row_index])
        print("beat_duration ", beat_duration)
        MyMIDI.addNote(0,0,midi_num,current_location,beat_duration,100)
        current_location += beat_duration
    
    current_location = 0
    for chord in chords:
        print("CHORD: ", chord)
        if chord < 12:
            notes = chord_template_major + chord + 48
        else:
            notes = chord_template_minor + chord + 48
        for note in notes:
            print("note: ", note)
            MyMIDI.addNote(1,0,int(note),current_location,num_time_sig,100)
        current_location += num_time_sig
    with open("bach.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file)
                              
write_midi(df_to_test, final_output)                         
        
        
        

        
#df_to_test
# track    = 0
# channel  = 0
# time     = 0    # In beats
# tempo    = 60   # In BPM
# volume   = 100  # 0-127, as per the MIDI standard

# MyMIDI = MIDIFile(1)  # One track, defaults to format 1 (tempo track is created
#                       # automatically)
# MyMIDI.addTempo(track, time, tempo)

# for i, pitch in enumerate(degrees):
#     MyMIDI.addNote(track, channel, pitch, time + i, duration, volume)

# with open("major-scale.mid", "wb") as output_file:
#     MyMIDI.writeFile(output_file)


Bachelors III.csv
3/4
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
mod  0.3333333333333333
beat_duration  1.3333333333333333
mod  0.3333333333333333
beat_duration  2.0
mod  0.3333333333333333
beat_du